<h1> Email Classifier </h1> 

In [1]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
#the spam and valid emails are stored in 2 folders called "easy_ham" and "spam"
#we want to read all the files in these 2 folders and pu the filenames into a list of spam and not spam
ham_filenames = [filename for filename in sorted(os.listdir(os.path.join(os.getcwd(),"easy_ham"))) if len(filename) > 20]
spam_filenames = [filename for filename in sorted(os.listdir(os.path.join(os.getcwd(),"spam"))) if len(filename) > 20]

In [3]:
len(ham_filenames)

2551

In [4]:
len(spam_filenames)

500

To parse emails we will use the email library of Python:

How the email library works for parsing:

The parser takes a serialized version of the email message(a stream of bytes) and converts it to a tree of EmailMessage objects.  The generator takes an EmailMessage and turns it back into a serialized byte stream.

There are 2 parser interfaces available, the Parser API and FeedParser API. The Parser API is most useful when you have the entire text of the message in memory or if the entire message lives in a file on the file system. FeedParser API is useful when you are reading the message from a stream which might block your waiting(reading from a url itself)

In [5]:
#collecting all the parsed ham messages
ham_path = os.path.join(os.getcwd(),"easy_ham")
spam_path = os.path.join(os.getcwd(),"spam")

In [6]:
import email
from email import policy


In [7]:
ham_messages = list()
for filename in ham_filenames:
    with open(os.path.join(ham_path,filename),'rb') as f:
        ham_messages.append(email.parser.BytesParser(policy = email.policy.default).parse(f))


In [8]:
spam_messages = list()
for filename in spam_filenames:
    with open(os.path.join(spam_path,filename),'rb') as f:
        spam_messages.append(email.parser.BytesParser(policy = email.policy.default).parse(f))

In [9]:
print(ham_messages[0].get_content().strip())

Date:        Wed, 21 Aug 2002 10:54:46 -0500
    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>
    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>


  | I can't reproduce this error.

For me it is very repeatable... (like every time, without fail).

This is the debug log of the pick happening ...

18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}
18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury
18:19:04 Ftoc_PickMsgs {{1 hit}}
18:19:04 Marking 1 hits
18:19:04 tkerror: syntax error in expression "int ...

Note, if I run the pick command by hand ...

delta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury
1 hit

That's where the "1 hit" comes from (obviously).  The version of nmh I'm
using is ...

delta$ pick -version
pick -- nmh-1.0.4 [compiled on fuchsia.cs.mu.OZ.AU at Sun Mar 17 14:55:56 

Emails can have different parts to it, with images, attachments. And these attachments can have emails in them. 

In [10]:
def get_email_structure(email):
    if isinstance(email,str):
        return email
    #get_payload() returns a list if the email is multipart and .is_multipart() = True
    payload = email.get_payload()
    if isinstance(payload,list):
        result = "multipart({})".format(', '.join([get_email_structure(sub_email) for sub_email in payload]))
        return result
    else:
        return email.get_content_type()

In [11]:
from collections import Counter

def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [12]:
structure_counts_ham = structures_counter(ham_messages)
structure_counts_spam = structures_counter(spam_messages)

In [13]:
structure_counts_ham.most_common()

[('text/plain', 2453),
 ('multipart(text/plain, application/pgp-signature)', 72),
 ('multipart(text/plain, text/html)', 8),
 ('multipart(text/plain, text/plain)', 4),
 ('multipart(text/plain)', 3),
 ('multipart(text/plain, application/octet-stream)', 2),
 ('multipart(text/plain, text/enriched)', 1),
 ('multipart(text/plain, application/ms-tnef, text/plain)', 1),
 ('multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature)',
  1),
 ('multipart(text/plain, video/mng)', 1),
 ('multipart(text/plain, multipart(text/plain))', 1),
 ('multipart(text/plain, application/x-pkcs7-signature)', 1),
 ('multipart(text/plain, multipart(text/plain, text/plain), text/rfc822-headers)',
  1),
 ('multipart(text/plain, multipart(text/plain, text/plain), multipart(multipart(text/plain, application/x-pkcs7-signature)))',
  1),
 ('multipart(text/plain, application/x-java-applet)', 1)]

In [14]:
structure_counts_spam.most_common()

[('text/plain', 221),
 ('text/html', 181),
 ('multipart(text/plain, text/html)', 45),
 ('multipart(text/html)', 19),
 ('multipart(text/plain)', 19),
 ('multipart(multipart(text/html))', 5),
 ('multipart(text/plain, image/jpeg)', 3),
 ('multipart(text/html, application/octet-stream)', 2),
 ('multipart(text/plain, application/octet-stream)', 1),
 ('multipart(text/html, text/plain)', 1),
 ('multipart(multipart(text/html), application/octet-stream, image/jpeg)', 1),
 ('multipart(multipart(text/plain, text/html), image/gif)', 1),
 ('multipart/alternative', 1)]

In [15]:
#creating a histogram of email types for spam and ham messages 
type_list = list()
temp = [[type_list.append(i) for i in structures] for structures in [structure_counts_ham, structure_counts_spam]]
ham_type_counts = [structure_counts_ham[i] for i in set(type_list)]
spam_type_counts = [structure_counts_spam[i] for i in set(type_list)]

In [16]:
type_counts_df = pd.DataFrame({'Email Type':list(set(type_list)), 'Ham Count':ham_type_counts, 'Spam Count':spam_type_counts})
type_counts_df

,Email Type,Ham Count,Spam Count
0,"multipart(multipart(text/plain, text/html), im...",0,1
1,"multipart(text/plain, multipart(text/plain, te...",1,0
2,"multipart(text/plain, application/octet-stream)",2,1
3,"multipart(multipart(text/plain, text/plain, te...",1,0
4,"multipart(text/plain, multipart(text/plain))",1,0
5,"multipart(text/plain, text/plain)",4,0
6,"multipart(text/plain, text/html)",8,45
7,"multipart(text/plain, multipart(text/plain, te...",1,0
8,multipart(text/html),0,19
9,"multipart(text/plain, application/x-pkcs7-sign...",1,0


Most valid(ham) emails are text/plain and contain a PGP(Pretty Good Privacy) signature, while Spam emails have a higher amount of HTML messages. 

In [17]:
#create a list of type of each email 
ham_email_type = [get_email_structure(email) for email in ham_messages]
spam_email_type = [get_email_structure(email) for email in spam_messages]

In [30]:
def FindEmailSender(email):
    try:
        return dict(email.items())['From']
    except:
        return "N/A"

In [31]:
#creating list of senders of each email
ham_email_senders = [FindEmailSender(email) for email in ham_messages]
spam_email_senders = [FindEmailSender(email) for email in spam_messages]

In [50]:
#combining the dataset to create a complete dataset for splitting into train and test sets
X = np.array(ham_messages + spam_messages, dtype = object)
y = np.array([0] * len(ham_messages) + [1]*len(spam_messages))

In [52]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, test_size=0.2)

In [100]:
#turning html in emails into tags we want
#turn head tags into ''
#turn anchor tags into hyper links
#turn all html tags into ''
import re
from html import unescape
def html_to_text(html):
    text = re.sub(r'<head.*?>.*?</head>','',html,flags = re.I|re.M|re.S)
    text = re.sub(r'<a.*?>.*?</a>',' HYPERLINK ',text,flags = re.I|re.M|re.S)
    text = re.sub(r'<.*?>','',text,flags=re.M|re.I|re.S)
    text = re.sub(r'(\s*\n)+','\n',text,flags = re.M|re.I|re.S)
    return unescape(text)

In [57]:
idx_html = [i for i in range(len(X_train)) if get_email_structure(X_train[i]) == 'text/html']

In [105]:
html_to_text(X_train[idx_html[5]].get_content().strip())


 HYPERLINK
Copyright 2002 - All rights reservedIf you would no longer like us
to contact you or feel that you havereceived this email in error,
please  HYPERLINK .


In [151]:
#check if any EmailMessage objects have more tha one text/html type occuring
html_check = list()
for j in range(len(X_train)):
    i = 0
    for part in X_train[j].walk():
        if part.get_content_type() == 'text/html':
            i += 1
        if i == 2:
            html_check.append(j)
    
html_check

[]

In [117]:
#turning emails to plain text

def email_to_text(email):
    

text/plain


In [2]:
os.getcwd()

'c:\\Users\\ASUS\\Desktop\\Hands on ML\\Hands-on-Machine-Learning-Textbook-Exercises\\DeepLearning_1\\SpamClassifier'